In [1]:
from pathlib import Path
import pandas as pd, numpy as np, pyarrow.parquet as pq, pyarrow as pa

In [2]:
raw_path = Path('../data_extraction/data/raw/2019/pbp_cfbd_raw.parquet')
mapped_path = Path('../data_extraction/data/raw/2019/pbp.parquet')

In [3]:
df_raw = pd.read_parquet(raw_path)
df_mapped = pd.read_parquet(mapped_path)

In [21]:
df_raw.columns

Index(['id', 'driveId', 'gameId', 'driveNumber', 'playNumber', 'offense',
       'offenseConference', 'offenseScore', 'defense', 'home', 'away',
       'defenseConference', 'defenseScore', 'period', 'offenseTimeouts',
       'defenseTimeouts', 'yardline', 'yardsToGoal', 'down', 'distance',
       'yardsGained', 'scoring', 'playType', 'playText', 'wallclock', 'ppa',
       'season', 'game_id', 'clock.seconds', 'clock.minutes',
       'passer_player_name', 'passer_player_id', 'completion',
       'receiver_player_name', 'receiver_played_id', 'rusher_player_name',
       'rusher_played_id', 'sack_yards', 'sacker_name', 'sacker_id', 'sack',
       'primary_pass_defender_name', 'primary_pass_defender_id',
       'interception', 'touchdown', 'fumble_forced_name', 'fumble_forced_id',
       'fumble', 'fumble_recovery_name', 'fumble_recovery_id', 'pass_breakup'],
      dtype='object')

In [22]:
df_mapped.columns

Index(['season', 'game_id', 'down', 'distance', 'yardline_100', 'yards_gained',
       'passer_player_id', 'passer_player_name', 'rusher_player_id',
       'rusher_player_name', 'receiver_player_id', 'receiver_player_name',
       'is_pass', 'is_rush', 'complete', 'touchdown', 'points_scored',
       'interception', 'sack', 'sack_yards'],
      dtype='object')

In [5]:
df_raw.sample(10)

,id,driveId,gameId,driveNumber,playNumber,offense,offenseConference,offenseScore,defense,home,...,primary_pass_defender_name,primary_pass_defender_id,interception,touchdown,fumble_forced_name,fumble_forced_id,fumble,fumble_recovery_name,fumble_recovery_id,pass_breakup
61781,401112161103849902,40111216115,401112161,15,1,Michigan State,Big Ten,21,Indiana,Michigan State,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
142954,401112513104937901,40111251320,401112513,20,11,NC State,ACC,20,Georgia Tech,Georgia Tech,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
13419,401135268101914602,4011352686,401135268,6,2,Marshall,Conference USA,0,UCF,Marshall,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
114968,401112495103914510,40111249521,401112495,21,8,Syracuse,ACC,20,Boston College,Syracuse,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
65437,401112108102946403,40111210813,401112108,13,2,Oklahoma State,Big 12,13,Kansas State,Oklahoma State,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
153803,401110866103878611,40111086614,401110866,14,10,Florida,SEC,30,Florida State,Florida,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
98642,401114357102869206,40111435711,401114357,11,4,Massachusetts,FBS Independents,14,UConn,Massachusetts,...,Omar Fortt,4239125,NaN,NaN,None,None,NaN,None,None,1.0
134096,401112219101874101,4011122191,401112219,1,6,Michigan,Big Ten,0,Michigan State,Michigan,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
15441,401112430101907604,4011124303,401112430,3,2,Wake Forest,ACC,7,Utah State,Wake Forest,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
75609,401117518104849905,40111751821,401117518,21,6,San Diego State,Mountain West,17,Colorado State,Colorado State,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN


In [6]:
df_mapped.sample(10)

,season,game_id,down,distance,yardline_100,yards_gained,passer_player_id,passer_player_name,rusher_player_id,rusher_player_name,receiver_player_id,receiver_player_name,is_pass,is_rush,complete,touchdown,points_scored,interception,sack,sack_yards
38070,2019,401112203,3,12,12,0,4037603,Josh Jackson,None,None,None,None,1,False,NaN,NaN,0,NaN,NaN,NaN
79743,2019,401117881,2,5,50,14,4044122,Anthony Russo,None,None,None,Branden Mack,1,False,1.0,NaN,0,NaN,NaN,NaN
30934,2019,401114229,2,29,94,4,None,None,None,Milo Hall,None,None,0,True,NaN,NaN,0,NaN,NaN,NaN
44802,2019,401110787,4,12,27,45,None,None,None,None,None,None,0,False,NaN,NaN,0,NaN,NaN,NaN
137954,2019,401119306,2,10,36,0,4046743,Grant Loy,None,None,None,Julian Ortega-Jones,1,False,NaN,NaN,0,NaN,NaN,NaN
25511,2019,401114254,3,5,25,4,4038741,Kato Nelson,None,None,None,Nate Stewart,1,False,1.0,NaN,0,NaN,NaN,NaN
48570,2019,401117506,4,7,23,40,None,None,None,None,None,None,0,False,NaN,NaN,0,NaN,NaN,NaN
76343,2019,401112194,4,18,75,17,None,None,None,None,None,None,0,False,NaN,NaN,0,NaN,NaN,NaN
94726,2019,401112484,1,10,75,0,4361653,Devin Leary,None,None,None,Devin Carter,1,False,NaN,NaN,0,NaN,NaN,NaN
12658,2019,401112073,2,9,9,6,None,None,None,Thomas Hutchings,None,None,0,True,NaN,NaN,0,NaN,NaN,NaN


In [36]:
# fumbles - quantitative data
df_raw[df_raw['fumble'].notna()][['gameId', 'offense', 'defense', 'period', 'clock.minutes', 'clock.seconds',
                                  'driveNumber', 'playNumber', 'offenseScore', 'defenseScore', 'down', 'distance',
                                 'yardsToGoal', 'yardsGained', 'fumble_forced_name', 'fumble_recovery_name', 'ppa']]

,gameId,offense,defense,period,clock.minutes,clock.seconds,driveNumber,playNumber,offenseScore,defenseScore,down,distance,yardsToGoal,yardsGained,fumble_forced_name,fumble_recovery_name,ppa
216,401112436,Notre Dame,Louisville,2,8,36,9,2,14,14,2,1,76,0,Khane Pass,None,-0.955868
323,401110732,Georgia,Vanderbilt,4,10,19,17,3,24,6,1,10,36,31,Feleti Afemui,Dashaun Jerkins,-3.989222
606,401110730,Tennessee,Georgia State,1,15,0,1,3,0,0,2,10,75,-2,Hardrick Willis,Ed Curney,-0.219275
854,401110724,Kentucky,Toledo,3,5,48,18,3,24,14,2,10,43,2,Saeed Holt,Nate Givhan,-2.708461
884,401110724,Kentucky,Toledo,4,3,30,28,2,38,17,2,16,61,6,Daniel Bolden,Dyontae Johnson,-0.472066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149444,401119310,Ohio,Akron,1,3,58,10,3,14,3,1,10,41,1,Dylan Meeks,A.J. Watts,-3.759480
151626,401117917,Tulane,SMU,1,13,14,2,3,0,0,1,10,63,14,Trevor Denbow,None,1.288180
153347,401119315,Toledo,Central Michigan,3,5,45,15,1,7,42,1,10,73,3,Kyron McKinnie-Harper,None,-0.320408
157856,401112121,Oklahoma State,Oklahoma,1,1,45,4,6,7,10,2,5,42,3,Parnell Motley,Parnell Motley,-3.484172


In [35]:
# fumbles - descriptions
for index, row in df_raw[df_raw['fumble'].notna()].head(10).iterrows():
    print(index)
    print(row['playType'])
    print(row['playText'])
for index, row in df_raw[df_raw['fumble'].notna()].tail(10).iterrows():
    print(index)
    print(row['playType'])
    print(row['playText'])

216
Fumble Recovery (Own)
Ian Book run for no gain to the NDame 24 Ian Book fumbled, forced by Khane Pass, recovered by NDame Jahmir Smith
323
Fumble Recovery (Opponent)
Jake Fromm pass complete to Kearis Jackson for 31 yds Kearis Jackson fumbled, forced by Feleti Afemui, recovered by Vandy Dashaun Jerkins  , return for 12 yds to the Vandy 17
606
Fumble Recovery (Opponent)
Ty Chandler run for a loss of 2 yards to the Tenn 23 Ty Chandler fumbled, forced by Hardrick Willis, recovered by GeoSt Ed Curney
854
Fumble Recovery (Opponent)
Terry Wilson pass complete to Lynn Bowden Jr. for 2 yds Lynn Bowden Jr. fumbled, forced by Saeed Holt, recovered by Toled Nate Givhan
884
Fumble Recovery (Opponent)
Terry Wilson pass complete to Christopher Rodriguez Jr. for 6 yds Christopher Rodriguez Jr. fumbled, forced by Daniel Bolden, recovered by Toled Dyontae Johnson  , return for 3 yds to the Kent 38
1716
Fumble Recovery (Opponent)
Abram Smith run for no gain to the Bayl 30 Abram Smith fumbled, forced

In [40]:
df_raw[df_raw['fumble_recovery_name'].notna()][['gameId', 'offense', 'defense', 'period', 'clock.minutes', 'clock.seconds',
                                  'driveNumber', 'playNumber', 'offenseScore', 'defenseScore', 'down', 'distance',
                                 'yardsToGoal', 'yardsGained', 'fumble_forced_name', 'fumble_recovery_name', 'ppa']]

,gameId,offense,defense,period,clock.minutes,clock.seconds,driveNumber,playNumber,offenseScore,defenseScore,down,distance,yardsToGoal,yardsGained,fumble_forced_name,fumble_recovery_name,ppa
323,401110732,Georgia,Vanderbilt,4,10,19,17,3,24,6,1,10,36,31,Feleti Afemui,Dashaun Jerkins,-3.989222
442,401110723,Florida,Miami,2,13,18,4,15,7,3,1,7,7,-2,None,Scott Patchan,-4.803245
446,401110723,Florida,Miami,2,10,46,6,2,7,3,2,7,37,-11,None,Shaquille Quarterman,-3.540925
591,401110730,Tennessee,Georgia State,4,8,56,17,5,23,28,1,10,49,-12,None,Jhi'Shawn Taylor,-3.119266
606,401110730,Tennessee,Georgia State,1,15,0,1,3,0,0,2,10,75,-2,Hardrick Willis,Ed Curney,-0.219275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152896,401110863,Ole Miss,Mississippi State,1,12,11,2,5,0,0,1,10,33,2,None,Brian Keith Cole,-4.096925
155127,401112525,Wake Forest,Syracuse,1,5,8,6,3,3,7,3,6,89,6,None,Andre Cisco,0.197546
155925,401117218,BYU,San Diego State,3,5,23,17,8,3,10,2,1,40,-10,None,Tariq Thompson,-3.385385
157288,401112200,Iowa,Nebraska,4,3,14,28,2,24,24,1,10,33,3,None,Cam Taylor,-4.096925


In [39]:
for index, row in df_raw[df_raw['fumble_recovery_name'].notna()].head(10).iterrows():
    print(index)
    print(row['playType'])
    print(row['playText'])
for index, row in df_raw[df_raw['fumble_recovery_name'].notna()].tail(10).iterrows():
    print(index)
    print(row['playType'])
    print(row['playText'])

323
Fumble Recovery (Opponent)
Jake Fromm pass complete to Kearis Jackson for 31 yds Kearis Jackson fumbled, forced by Feleti Afemui, recovered by Vandy Dashaun Jerkins  , return for 12 yds to the Vandy 17
442
Fumble Recovery (Opponent)
Feleipe Franks run for a loss of 2 yards to the MiaFl 9 Feleipe Franks fumbled, recovered by MiaFl Scott Patchan
446
Fumble Recovery (Opponent)
Malik Davis run for a loss of 11 yards to the MiaFl 48 Malik Davis fumbled, recovered by MiaFl Shaquille Quarterman
591
Sack
Jarrett Guarantano sacked by Evan Jones for a loss of 12 yards to the Tenn 39 Jarrett Guarantano fumbled, recovered by GeoSt Jhi'Shawn Taylor , return for 0 yards
606
Fumble Recovery (Opponent)
Ty Chandler run for a loss of 2 yards to the Tenn 23 Ty Chandler fumbled, forced by Hardrick Willis, recovered by GeoSt Ed Curney
854
Fumble Recovery (Opponent)
Terry Wilson pass complete to Lynn Bowden Jr. for 2 yds Lynn Bowden Jr. fumbled, forced by Saeed Holt, recovered by Toled Nate Givhan
884
F

In [43]:
df_raw[np.logical_or(df_raw['fumble_recovery_name'].notna(), df_raw['fumble'].notna())][['gameId', 'offense', 'defense', 
                                                                                         'period', 'clock.minutes', 'clock.seconds',
                                  'driveNumber', 'playNumber', 'offenseScore', 'defenseScore', 'down', 'distance',
                                 'yardsToGoal', 'yardsGained', 'fumble_forced_name', 'fumble_recovery_name', 'ppa']]

,gameId,offense,defense,period,clock.minutes,clock.seconds,driveNumber,playNumber,offenseScore,defenseScore,down,distance,yardsToGoal,yardsGained,fumble_forced_name,fumble_recovery_name,ppa
216,401112436,Notre Dame,Louisville,2,8,36,9,2,14,14,2,1,76,0,Khane Pass,None,-0.955868
323,401110732,Georgia,Vanderbilt,4,10,19,17,3,24,6,1,10,36,31,Feleti Afemui,Dashaun Jerkins,-3.989222
442,401110723,Florida,Miami,2,13,18,4,15,7,3,1,7,7,-2,None,Scott Patchan,-4.803245
446,401110723,Florida,Miami,2,10,46,6,2,7,3,2,7,37,-11,None,Shaquille Quarterman,-3.540925
591,401110730,Tennessee,Georgia State,4,8,56,17,5,23,28,1,10,49,-12,None,Jhi'Shawn Taylor,-3.119266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155127,401112525,Wake Forest,Syracuse,1,5,8,6,3,3,7,3,6,89,6,None,Andre Cisco,0.197546
155925,401117218,BYU,San Diego State,3,5,23,17,8,3,10,2,1,40,-10,None,Tariq Thompson,-3.385385
157288,401112200,Iowa,Nebraska,4,3,14,28,2,24,24,1,10,33,3,None,Cam Taylor,-4.096925
157856,401112121,Oklahoma State,Oklahoma,1,1,45,4,6,7,10,2,5,42,3,Parnell Motley,Parnell Motley,-3.484172
